<div class="alert alert-block alert-info">
<h3>Student Information</h3> Please provide information about yourself.<br>
<b>Name</b>: Avanrika Yellapantula<br>
<b>NetID</b>: ay257<br>
<b>Recitation (01/02)</b>: 01<br>
<b>Notes to Grader</b> Sometimes the code for the Part 3 actual/true loss works and sometimes it gives an error on dimensionality:<br>
<br><br>
<b>IMPORTANT</b>
Your work will not be graded withour your initials below<br>
I certify that this lab represents my own work and I have read the RU academic intergrity policies at<br>
<a href="https://www.cs.rutgers.edu/academic-integrity/introduction">https://www.cs.rutgers.edu/academic-integrity/introduction </a><br>
<b>Initials</b>:    AY 


<h3>Grader Notes</h3>
<b>Your Grade<b>:<br>
<b>Grader Initials</b>:<br>
<b>Grader Comments</b> (optional):<br>
</div>

# Lab 7: Linear Models, Feature Engineering and Logistic Regression

** This lab is due Monday April 22, 2019 at 11:59pm (graded on accuracy and completeness) **

In this lab we will work through the process of:
1. implementing a linear model, defining loss functions, 
2. feature engineering,
3. minimizing loss functions using numeric libraries 
4. how to implement logistic regression model for a binary classification problem

We will use the toy tip calculation dataset from sns.

## Set up

In [33]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
np.random.seed(42)
plt.style.use('fivethirtyeight')
sns.set()
sns.set_context("talk")
%matplotlib inline

## Load the Tips Dataset

To begin with, we load the tips dataset from the `seaborn` library.  The tips data contains records of tips, total bill, and information about the person who paid the bill.

In [34]:
data = sns.load_dataset("tips")
print("Number of Records:", len(data))
data.head(7)

Number of Records: 244


,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
5,25.29,4.71,Male,No,Sun,Dinner,4
6,8.77,2.00,Male,No,Sun,Dinner,2


# Part 1: Defining the Model and Feature Engineering

In lab 6, we defined a simple linear model with only one parameter. Now let's make a more complicated model that utilizes other features in a dataset. Let our prediction for tip be a combination of the following features:

$$
\text{Tip} = \theta_1 * \text{total_bill} + \theta_2 * \text{sex} + \theta_3 * \text{smoker} + \theta_4 * \text{day} + \theta_5 * \text{time} + \theta_6 * \text{size}
$$

Notice that some of these features are not numbers! But our linear model will need to predict a numerical value. 
### Task 1.1 Split the data into Y(tip) and X(all others)

In [35]:
## BEGIN SOLUTION
Y = data[['tip']]
X = data[['total_bill','sex','smoker','day','time','size']]
## END SOLUTION

## Task 1.2: Feature Engineering
First, let's convert everything to numerical values. A straightforward approach is to map some of these non-numerical features into numerical ones. For example, we can treat the day as a value from 1-7. However, one of the disadvantages in directly translating to a numeric value is that we unintentially assign certain features disproportionate weight. Consider assigning Sunday to the numeric value of 7. Monday is assigned to 1 and thus Sunday has 7 times the influence of Monday in our linear model which can lower the accuracy of our model.

solution: one-hot encoding! 

As discussed in lecture 9.2, one-hot encoding will produce a binary vector indicating the non-numeric feature. Sunday would be encoded as a [0 0 0 0 0 0 1]. This assigns a more even weight across each category in non-numeric features. Complete the code below to one-hot encode our dataset.

In [36]:
def one_hot_encode(data):
    """
    Return the one-hot encoded dataframe of our input, data. 
    Hint: check pd.get_dummies
    """
    ### BEGIN SOLUTION
    return pd.get_dummies(data)
    ### END SOLUTION

In [37]:
one_hot_X = one_hot_encode(X)
one_hot_X.head()

,total_bill,size,sex_Male,sex_Female,smoker_Yes,smoker_No,day_Thur,day_Fri,day_Sat,day_Sun,time_Lunch,time_Dinner
0,16.99,2,0,1,0,1,0,0,0,1,0,1
1,10.34,3,1,0,0,1,0,0,0,1,0,1
2,21.01,3,1,0,0,1,0,0,0,1,0,1
3,23.68,2,1,0,0,1,0,0,0,1,0,1
4,24.59,4,0,1,0,1,0,0,0,1,0,1


## Task 1.2: Defining the Model
Now that all of our data is numeric, let's define our model function. Note that X and thetas are matrices now. Use matrix products to compute the model.

$$
\text{Tip} = \theta_1 * \text{total_bill} + \theta_2 * \text{sex} + \theta_3 * \text{smoker} + \theta_4 * \text{day} + \theta_5 * \text{time} + \theta_6 * \text{size}
$$


In [38]:
def linear_model(thetas, X):
    """
    Return the linear combination of thetas and features as defined above.
    """
    ### BEGIN SOLUTION
#     tippy=[]
    return sum(thetas*X)
    
#     return for gg in thetas: for kk in X: gg*kk
    ### END SOLUTION
#     linear_model(np.arange(1,5), np.arange(1,5))

In [39]:
assert linear_model(np.arange(1,5), np.arange(1,5)) == 30

# Part 2: Fitting the Model: Numeric Methods
Recall in the previous lab we defined multiple loss functions and found optimal theta using the scipy.minimize function. Adapt the loss functions and optimization code from the previous lab (provided below) to work with your new linear model.

In [46]:
from scipy.optimize import minimize

def abserror(y, y_hat):
    return np.abs(y-y_hat)

def sqerror(y, y_hat):
    return (y - y_hat)**2

def minimize_average_loss(loss_function, model, x, y):
    """
    loss_function: either the squared or absolute loss functions from above.
    model: the model (as defined above)
    x: the x values (one-hot encoded data)
    y: the y values (tip amounts)
    return the estimate for each theta as a vector
    
    Note we will ignore failed convergence for this lab ... 
    """
    """
    Notes on the following function call which you need to finish:
     
     0. the ... should be replaced with the average loss evaluated on 
           the data x, y using the model and appropriate loss function
     1. x0 are the initial values for THETA.  Yes, this is confusing
           but optimization people like x to be the thing they are 
           optimizing.
     2. We extract the 'x' entry in the dictionary which corresponds
           to the value of thetas at the optimum
    """
    ### BEGIN SOLUTION
#     scipy.optimize.minimize(fun, x0, args=(), method=None)
    return minimize(linear_model,one_hot_X,method='BFGS')
    ### END SOLUTION
    ## return minimize(lambda theta: ..., x0=...)

minimize_average_loss(sqerror, linear_model, one_hot_X, Y)

TypeError: linear_model() missing 1 required positional argument: 'X'

# Part 3: Fitting the Model: Analytic Methods
Let's also fit our model analytically, for the l2 loss function. In this question we will derive an analytical solution, fit our model and compare our results with our numerical optimization results.

## Task 3.1: Least Squares Solution
Recall that if we're fitting a linear model with the l2 loss function, we are performing least squares! Remember, we are solving the following optimization problem for least squares:

$$\min_{\theta} ||X\theta - y||^2$$

Let's begin by deriving the analytic solution to least squares. Write your answer in LaTeX in the cell below. Assume X is full column rank. You are given step 1 to get things started. You need to complete the steps 2-4 where step 4 shows the value of theta that optimizes given X and y.

### BEGIN SOLUTION
Take the gradient and set theta equal to 0. 

$$2(X\theta - y) = 0$$
$$ step 2$$
$$ step 3$$
$$ \theta = step 4$$
### END SOLUTION

## Task 3.2: Solving for Theta
Using your formuala above, let us find the analytic solution for $\theta$. That is, the optimal numerical thetas for our tips dataset. Fill out the function below. Make sure you use the float type in your calculations using .astype(float) and use the np.linalg.inv function.

In [125]:
from numpy.linalg import inv
def get_analytical(x, y):
    """
    x: our one-hot encoded dataset
    y: tip amounts
    """
    ### BEGIN SOLUTION
    one_hot_X_t = one_hot_X.transpose()
#     one_X = one_hot_X_t*one_hot_X
#     print(one_hot_X_t) 12x244
#     print(one_hot_X)   244x12
    one_X = one_hot_X_t.dot(one_hot_X)
#     must take inverse
    one_X_inv = pd.DataFrame(np.linalg.pinv(one_X.values), one_X.columns, one_X.index)
    one_X_mul = one_X_inv.dot(one_hot_X_t)
    analytical_t = one_X_mul.dot(Y)   
    analytical_t = analytical_t.values
#     print(analytical_t)
    return analytical_t
    

    ### END SOLUTION
get_analytical(one_hot_X,Y)

array([[0.09448701],
       [0.175992  ],
       [0.18411021],
       [0.21655115],
       [0.15712652],
       [0.24353484],
       [0.0152057 ],
       [0.1774649 ],
       [0.05600652],
       [0.15198424],
       [0.23439498],
       [0.16626638]])

In [129]:
analytical_thetas = get_analytical(one_hot_X.astype(float), Y.astype(float))
print("Our analytical loss is: ", sqerror(linear_model(analytical_thetas, one_hot_X),Y).mean())
print("Our numerical loss is: ", sqerror(linear_model(minimize_average_loss(sqerror, linear_model, one_hot_X, tips), one_hot_X), Y).mean())

ValueError: Unable to coerce to DataFrame, shape must be (244, 12): given (12, 1)

In [127]:
assert np.isclose(sqerror(linear_model(analytical_thetas, one_hot_X),Y).mean(), 59.733414754098362)

ValueError: Unable to coerce to DataFrame, shape must be (244, 12): given (12, 1)

## Task 3.3: Weird Results?
Our analytical loss is surprisingly much worse than our numerical loss. Why is this? 

Hint: https://stackoverflow.com/questions/31256252/why-does-numpy-linalg-solve-offer-more-precise-matrix-inversions-than-numpy-li

### BEGIN SOLUTION
The theoretical loss is 59.733414754098355, while the actual loss is 1.0103535612368402.
The theoretical loss is based off of the inverse function for numpy which np.linalg.pinv()... This will not use back substitution to solve for the inverse. Instead it will use the identity and solve for the following: A^(-1)*A=I


### END SOLUTION

# Part 4 - Logistic Regression
In this part we will implement a logistic regression model with the tip data set. We will use Male and Female as the binary classification. We will use the LogisiticRegression model from sklearn to find a prediction model.

## task 4.1
** create the X (one-hot features) and y (sex) **

In [131]:
## BEGIN SOLTUION
X = pd.get_dummies(data[['total_bill','smoker','day','time','size']])
y = data[['sex']]
"""
Y = data[['tip']]
X = data[['total_bill','sex','smoker','day','time','size']]
pd.get_dummies(data)
"""

## END SOLUTION
print(X.head())
print(y.head())


   total_bill  size  smoker_Yes  smoker_No  day_Thur  day_Fri  day_Sat  \
0       16.99     2           0          1         0        0        0   
1       10.34     3           0          1         0        0        0   
2       21.01     3           0          1         0        0        0   
3       23.68     2           0          1         0        0        0   
4       24.59     4           0          1         0        0        0   

   day_Sun  time_Lunch  time_Dinner  
0        1           0            1  
1        1           0            1  
2        1           0            1  
3        1           0            1  
4        1           0            1  
      sex
0  Female
1    Male
2    Male
3    Male
4  Female


## task 4.2 
Split the data set to training and test using sklearn train_test_split

In [132]:
from sklearn.model_selection import train_test_split
## BEGIN SOLUTION
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
## END SOLUTION

print (X_train.head(), X_test.head(), y_train.head(), y_test.head())

     total_bill  size  smoker_Yes  smoker_No  day_Thur  day_Fri  day_Sat  \
114       25.71     3           0          1         0        0        0   
65        20.08     3           0          1         0        0        1   
132       11.17     2           0          1         1        0        0   
207       38.73     4           1          0         0        0        1   
162       16.21     3           0          1         0        0        0   

     day_Sun  time_Lunch  time_Dinner  
114        1           0            1  
65         0           0            1  
132        0           1            0  
207        0           0            1  
162        1           0            1        total_bill  size  smoker_Yes  smoker_No  day_Thur  day_Fri  day_Sat  \
24        19.82     2           0          1         0        0        1   
6          8.77     2           0          1         0        0        0   
153       24.55     4           0          1         0        0        0   

## task 4.3 logistic regression model
Fit a logisitic regression model using LogisticRegression from sklearn

In [136]:
from sklearn.linear_model import LogisticRegression
## BEGIN SOLUTION
logmodel = LogisticRegression().fit(X, y)
## END SOLUTION

C:\Users\avocado\AppData\Local\Programs\Python\Python37-32\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\avocado\AppData\Local\Programs\Python\Python37-32\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


## task 4.4 Predicting
Now we can use the logmodel from previous part to predict on the test data. Read documentation to see how. It should be just a one line of code.

In [142]:
## BEGIN SOLUTION
predictions = logmodel.predict(X_test)
## END SOLUTION

## task 4.5 reporting
use classification_report from sklearn.metrics to report the accuracy of the model.

In [148]:
from sklearn.metrics import classification_report
## BEGIN SOLUTION
visualizer = classification_report(X, y)
## END SOLUTION

ValueError: Classification metrics can't handle a mix of continuous-multioutput and binary targets

### Feedback
Please provide feedback on this lab.
* how would you rate this lab (from 1-10, 10-highest) :
* how can we improve his lab? :

<div class="alert alert-block alert-info">
<h2>Submission Instructions</h2> 
<b> File Name:</b> Please name the file as your_section_your_netID_lab7.jpynb<br>
<b> Submit To: </b> Canvas &rarr; Assignments &rarr; lab7 <br>
<b>Warning:</b> Failure to follow directions may result in loss points.<br>
</div>

Developed by A.D. Gunawardena. Credits: Josh Hug, Berkeley Data Science Lab